In [1]:
import pandas as pd
import os
import csv
import json
from collections import defaultdict

In [2]:
# Lista dei file di ranking per ciascun decennio e per il ranking attuale
file_paths = [
    ('../data/ranking/atp_rankings_70s.csv', 'ranking_70s'),
    ('../data/ranking/atp_rankings_80s.csv', 'ranking_80s'),
    ('../data/ranking/atp_rankings_90s.csv', 'ranking_90s'),
    ('../data/ranking/atp_rankings_00s.csv', 'ranking_00s'),
    ('../data/ranking/atp_rankings_10s.csv', 'ranking_10s'),
    ('../data/ranking/atp_rankings_20s.csv', 'ranking_20s'),
]

In [11]:
def clean_data(file_path, label):
    # Funzione per convertire la data
    def extract_date_components(date_str):
        year = date_str[:4]
        month = date_str[4:6]
        day = date_str[6:]
        return year, month, day

    # Leggi il CSV e costruisci la struttura annidata
    rankings_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    # Apri il file CSV e leggi i dati
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            date = row['ranking_date']
            rank = int(row['rank'])
            
            # Considera solo le prime 100 posizioni
            if rank > 100:
                continue  # Salta questa riga se il ranking è superiore a 100
            
            player = row['player']
            
            # Controlla se il campo 'points' è vuoto
            points_str = row['points']
            points = 0  # Imposta un valore di default
            if points_str != '':  # Se il campo non è vuoto, convertilo in int
                try:
                    points = int(points_str)
                except ValueError:
                    print(f"Warning: Non è possibile convertire '{points_str}' in int per il player {player}")
            
            year, month, day = extract_date_components(date)
            
            # Aggiungi le informazioni nella struttura annidata
            rankings_data[year][month][day].append({
                'rank': rank,
                'player': player,
                'points': points
            })

    # Filtra i dati per prendere solo il primo giorno disponibile di ogni mese
    filtered_rankings = defaultdict(lambda: defaultdict(dict))

    for year, months in rankings_data.items():
        for month, days in months.items():
            # Trova il primo giorno disponibile ordinando i giorni
            first_day = sorted(days.keys())[0]
            # Salva i dati del primo giorno disponibile
            filtered_rankings[year][month] = days[first_day]

    label = label + '_clean.json'

    json_path = os.path.join('.', 'ranking_json', label)

    # Converti la struttura filtrata in un file JSON
    with open(json_path, 'w', encoding='utf-8') as jsonfile:
        json.dump(filtered_rankings, jsonfile, ensure_ascii=False, indent=4)

    print("Conversione completata e file JSON filtrato creato.")

In [12]:

for file_path, label in file_paths:
    clean_data(file_path, label);

Conversione completata e file JSON filtrato creato.
Conversione completata e file JSON filtrato creato.
Conversione completata e file JSON filtrato creato.
Conversione completata e file JSON filtrato creato.
Conversione completata e file JSON filtrato creato.
Conversione completata e file JSON filtrato creato.
